In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'DBLP-ACM'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.08164518, -0.14257213,  0.00138122, -0.15239411, -0.15116636,
        -0.12062615, -0.08609576,  0.10328422,  0.01135666, -0.14947821],
       [-0.13081761,  0.08238994,  0.01865828, -0.07190776, -0.02767296,
        -0.0442348 ,  0.16813417,  0.19266247, -0.14633124,  0.11719078],
       [-0.13977055,  0.08413002,  0.16711281, -0.03499044,  0.14971319,
         0.01510516,  0.13116635, -0.01300191,  0.15946463, -0.10554493],
       [ 0.10737111, -0.06369834,  0.10715807, -0.17234768,  0.12611845,
        -0.0556029 ,  0.13101832,  0.06327226, -0.08713251,  0.08628036],
       [ 0.13265514, -0.08972533, -0.10050865, -0.17843337, -0.13062055,
         0.10193286,  0.0197355 , -0.14974568,  0.00467955, -0.09196338]]), array([[ 0.0770881 ,  0.14130435, -0.12771739,  0.08052059,  0.13200801,
        -0.09825515,  0.11999428, -0.06521739, -0.12628719,  0.03160755],
       [-0.09193112,  0.10416963,  0.139889  , -0.05663868, -0.11000427,
        -0.07898107, -0.13078127,  0.06873

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


2
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(10):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.9594594594594594
K: 1, L: 2, Precision:0.9954954954954955
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.9256756756756757
K: 2, L: 2, Precision:0.9932432432432432
K: 2, L: 3, Precision:1.0
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.9054054054054054
K: 3, L: 2, Precision:0.9774774774774775
K: 3, L: 3, Precision:1.0
K: 3, L: 4, Precision:0.9977477477477478
K: 3, L: 5, Precision:1.0
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.8400900900900901
K: 4, L: 2, Precision:0.972972972972973
K: 4, L: 3, Precision:0.9842342